<a href="https://colab.research.google.com/github/maceteligolden/modelinganduncertaining/blob/main/modelandoptimisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install bertopic hdbscan umap sentence_transformers

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nv

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import nltk
import json
import glob
import numpy as np
import gensim
import gensim.corpora as corpora
import gensim.utils as simple_preprocess
import gensim.models as models
import matplotlib.pyplot as plt
import spacy
from nltk.corpus import stopwords
import pyLDAvis
import pyLDAvis.gensim
import pandas as pd

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# constant and global initialization
stopwords = stopwords.words('english')

base_url = "/content/drive/MyDrive/twitter_dataset.csv"

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
# Fetch corpus
def fetch_data_set(file):
  return pd.read_csv(file)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
# fetch dataset from location
df = fetch_data_set(base_url)

tweets = df['Text']

df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Tweet_ID,Username,Text,Retweets,Likes,Timestamp
0,1,julie81,Party least receive say or single. Prevent pre...,2,25,2023-01-30 11:00:51
1,2,richardhester,Hotel still Congress may member staff. Media d...,35,29,2023-01-02 22:45:58
2,3,williamsjoseph,Nice be her debate industry that year. Film wh...,51,25,2023-01-18 11:25:19
3,4,danielsmary,Laugh explain situation career occur serious. ...,37,18,2023-04-10 22:06:29
4,5,carlwarren,Involve sense former often approach government...,27,80,2023-01-24 07:12:21


In [6]:
# lemmitize tweets
def lemmatize_token_pos(tweets):
  pos_tags = ["NOUN", "ADJ", "VERB", "ADV"]
  nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
  tweets_out = []
  for tweet in tweets:
    doc = nlp(tweet)
    new_tweet = []
    for token in doc:
      if token.pos_ in pos_tags:
        new_tweet.append(token.lemma_)
    tweets_out.append(" ".join(new_tweet))
  return (tweets_out)

# apply preprocessing methods
def preprocess(text):
  final = []
  for tweet in tweets:
    new = gensim.utils.simple_preprocess(tweet, deacc=True)
    final.append(new)
  return (final)

# generate words
def gen_words(tweets):
  lemmatized_tweets = lemmatize_token_pos(tweets)
  print(f"Sample Lemmatized tweets: {lemmatized_tweets[0][0:90]}")
  preprocessed_tweets = preprocess(lemmatized_tweets)
  print(f"Sample Preprocessed tweets: {preprocessed_tweets[0][0:90]}")
  return (preprocessed_tweets)

words = gen_words(tweets)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Sample Lemmatized tweets: least receive say single prevent prevent husband affect cup style evening protect effect s
Sample Preprocessed tweets: ['party', 'least', 'receive', 'say', 'or', 'single', 'prevent', 'prevent', 'husband', 'affect', 'may', 'himself', 'cup', 'style', 'evening', 'protect', 'effect', 'another', 'themselves', 'stage', 'perform', 'possible', 'try', 'tax', 'share', 'style', 'television', 'with', 'successful', 'much', 'sell', 'development', 'economy', 'effect']


In [7]:
id2word = corpora.Dictionary(words)

# filter extremes
id2word.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

corpus = []
for word in words:
  new = id2word.doc2bow(word)
  corpus.append(new)

tfidf_transformer = models.TfidfModel(corpus, id2word=id2word)

# Transform the corpus using the TF-IDF model
corpus_tfidf = tfidf_transformer[corpus]

word = id2word[[0][:1][0]]
print(word)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Dictionary<969 unique tokens: ['affect', 'another', 'cup', 'development', 'economy']...>
[(0, 0.15620741835387061), (1, 0.16190123510414275), (2, 0.1588987670602643), (3, 0.1601621037106702), (4, 0.16279557032428305), (5, 0.3172451657107869), (6, 0.15834801377206476), (7, 0.16463702906543748), (8, 0.15491746965136555), (9, 0.1560768169360203), (10, 0.1595964200070035), (11, 0.15903747079317418), (12, 0.1547904027823514), (13, 0.15848509760504886), (14, 0.1588987670602643), (15, 0.15848509760504886), (16, 0.3172451657107869), (17, 0.15726551788180204), (18, 0.15353829712815825), (19, 0.16044752209384655)]
affect


In [8]:
# train LDA model
lda_model = gensim.models.ldamodel.LdaModel(
    corpus=corpus_tfidf,
    id2word=id2word,
    num_topics=5,
    random_state=100,
    update_every=1,
    chunksize=100,
    passes=10,
    alpha="auto",
  )

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
# display an Intertopic distance map
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus_tfidf, id2word, mds="mds", R=30)
vis

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.181812 -0.037822       1        1  36.126698
2     -0.228316 -0.176689       2        1  29.331214
4     -0.048088  0.264334       3        1  19.998606
0      0.284721 -0.232520       4        1   9.167647
1      0.173495  0.182697       5        1   5.375834, topic_info=           Term       Freq      Total Category  logprob  loglift
901         add  73.000000  73.000000  Default  30.0000  30.0000
376  conference  73.000000  73.000000  Default  29.0000  29.0000
143    research  72.000000  72.000000  Default  28.0000  28.0000
274   direction  71.000000  71.000000  Default  27.0000  27.0000
146       sense  66.000000  66.000000  Default  26.0000  26.0000
..          ...        ...        ...      ...      ...      ...
237        sort  56.481447  57.293311   Topic5  -3.9954   2.9090
335     measure  56.238715  57.050359   Topic5  -3.9997   2.9089
261         she  56.166925  56.978056   Topic5  -4.0010   2.9089
229        most  56.011987  56.823738   Topic5  -4.0038   2.9089
902        edge  55.882345  56.693340   Topic5  -4.0061   2.9088

[186 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
807       4  0.985906  ability
901       5  0.986772      add
31        3  0.980099   agency
713       1  0.986588    agent
722       3  0.993553    agree
...     ...       ...      ...
57        3  0.981571    which
622       1  0.981519    whose
837       4  0.984608   within
911       2  0.986945    write
87        3  0.982689     yard

[162 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 5, 1, 2])

In [10]:
from gensim.models import CoherenceModel

# coherence model of lda
coherence_model_lda = CoherenceModel(model=lda_model, texts=words, dictionary=id2word, coherence='c_v')

# calculate coherence score for lda
coherence_score = coherence_model_lda.get_coherence()

print(f'Coherence Score of LDA: {coherence_score}')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Coherence Score: 0.21557482199000555


In [13]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import umap
import hdbscan

# extract BERT embeddings
topic_model = BERTopic(embedding_model="all-MiniLM-L6-v2")

# transform tweets to topics
topics, probs = topic_model.fit_transform(tweets)

# get topic information
topics_info = topic_model.get_topic_info()

# save topic to a data frame and display it on a table
bertdf = pd.DataFrame({"topic": topics, "document": tweets})
bertdf.head()



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,topic,document
0,0,Party least receive say or single. Prevent pre...
1,0,Hotel still Congress may member staff. Media d...
2,0,Nice be her debate industry that year. Film wh...
3,0,Laugh explain situation career occur serious. ...
4,0,Involve sense former often approach government...


In [45]:
# show bar chart of generate topics and words associated to those topics
topic_model.visualize_barchart()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [16]:
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary

topics_list = []
for topic in range(len(topic_model.get_topic_info()) - 1):

    topic_words = [word for word, _ in topic_model.get_topic(topic)]
    topics_list.append(topic_words)

id2word = Dictionary(words)

coherence_model = CoherenceModel(
    topics=topics_list,
    texts=words,
    dictionary=id2word,
    coherence='c_v'
)

coherence_score = coherence_model.get_coherence()

print(f'Coherence Score for BERTopic model: {coherence_score}')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Coherence Score: 0.341871593280358
